In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy import signal
import json
import os
from pathlib import Path
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict
from enum import Enum

# Set matplotlib to display in notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("All dependency libraries imported successfully!")

所有依赖库已成功导入！


In [ ]:
# Waveform type enumeration
class WaveformType(Enum):
    SINE = "sine"
    COSE = "cose"
    IQ = "iq"
    MULTI_TONE = "multi_tone"
    SQUARE = "square"
    PULSE = "pulse"
    TRIANGLE = "triangle"
    CHIRP = "chirp"

# Signal parameter configuration
@dataclass
class SignalParams:
    waveform_type: str
    duration: float = 1.0
    sample_rate: int = 44100
    amplitude: float = 1.0
    frequency: float = 1000.0
    phase: float = 0.0
    dc_offset: float = 0.0
    
    # Square wave parameters
    duty_cycle: float = 0.5
    
    # Multi-tone signal parameters
    frequencies: List[float] = None
    amplitudes: List[float] = None
    phases: List[float] = None
    
    # Pulse signal parameters
    pulse_width: float = 0.1
    pulse_period: float = 1.0
    
    # Linear chirp signal parameters
    f0: float = 100.0
    f1: float = 1000.0
    method: str = "linear"

print("Signal parameter class definition completed!")

信号参数类定义完成！


In [ ]:
# Jupyter optimized signal generator class
class JupyterSignalGenerator:
    """Signal generator optimized for Jupyter notebook"""
    
    def __init__(self):
        self.DATA_TYPE_MAP = {
            0: np.uint32, 1: np.int32, 2: np.uint8, 3: np.int8,
            4: np.uint16, 5: np.int16, 6: np.int16, 7: np.float32,
            9: np.uint64, 11: np.float64,
        }
        self.generated_signals = {}
        
    def generate_sine_wave(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate sine wave"""
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        signal_data = params.amplitude * np.sin(2 * np.pi * params.frequency * t + params.phase) + params.dc_offset
        return t, signal_data

    def generate_cose_wave(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate cosine wave"""
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        signal_data = params.amplitude * np.cos(2 * np.pi * params.frequency * t + params.phase) + params.dc_offset
        return t, signal_data

    def generate_IQ_wave(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate IQ waveform with real sine and imaginary cosine"""
        # Generate time array
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        
        # Generate I component (real part, cosine)
        signal_data_I = params.amplitude * np.sin(2 * np.pi * params.frequency * t + params.phase) + params.dc_offset
        
        # Generate Q component (imaginary part, sine)
        signal_data_Q = params.amplitude * np.cos(2 * np.pi * params.frequency * t + params.phase) + params.dc_offset
        
        # Interleave I and Q components into IQ waveform [I0, Q0, I1, Q1, I2, Q2, ...]
        signal_data_IQ = np.zeros(len(t) * 2)
        signal_data_IQ[::2] = signal_data_I  # Even positions for I component
        signal_data_IQ[1::2] = signal_data_Q  # Odd positions for Q component
        
        # Create corresponding time array for IQ waveform
        t_IQ = np.linspace(0, params.duration, len(signal_data_IQ), endpoint=False)
        
        return t_IQ, signal_data_IQ
    
    def generate_multi_tone(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate multi-tone signal"""
        if not params.frequencies:
            raise ValueError("Multi-tone signal requires frequency list")
        
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        
        num_tones = len(params.frequencies)
        amplitudes = params.amplitudes if params.amplitudes else [params.amplitude] * num_tones
        phases = params.phases if params.phases else [0.0] * num_tones
        
        # Ensure list lengths are consistent
        amplitudes = amplitudes[:num_tones] + [amplitudes[-1]] * (num_tones - len(amplitudes))
        phases = phases[:num_tones] + [phases[-1]] * (num_tones - len(phases))
        
        signal_data = np.zeros_like(t)
        for freq, amp, phase in zip(params.frequencies, amplitudes, phases):
            signal_data += amp * np.sin(2 * np.pi * freq * t + phase)
        
        signal_data += params.dc_offset
        return t, signal_data
    
    def generate_square_wave(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate square wave"""
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        signal_data = params.amplitude * signal.square(2 * np.pi * params.frequency * t + params.phase, duty=params.duty_cycle) + params.dc_offset
        return t, signal_data
    
    def generate_pulse_signal(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate pulse signal"""
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        signal_data = np.zeros_like(t) + params.dc_offset
        
        pulse_samples = int(params.pulse_width * params.sample_rate)
        period_samples = int(params.pulse_period * params.sample_rate)
        
        for i in range(0, len(t), period_samples):
            end_idx = min(i + pulse_samples, len(t))
            signal_data[i:end_idx] = params.amplitude + params.dc_offset
        
        return t, signal_data
    
    def generate_triangle_wave(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate triangle wave"""
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        signal_data = params.amplitude * signal.sawtooth(2 * np.pi * params.frequency * t + params.phase, width=0.5) + params.dc_offset
        return t, signal_data
    
    def generate_chirp_signal(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate linear chirp signal"""
        t = np.linspace(0, params.duration, int(params.sample_rate * params.duration), endpoint=False)
        signal_data = params.amplitude * signal.chirp(t, params.f0, params.duration, params.f1, method=params.method, phi=params.phase) + params.dc_offset
        return t, signal_data

    def generate_signal(self, params: SignalParams) -> Tuple[np.ndarray, np.ndarray]:
        """Generate signal based on parameters"""
        waveform_type = WaveformType(params.waveform_type)
        
        if waveform_type == WaveformType.SINE:
            return self.generate_sine_wave(params)
        elif waveform_type == WaveformType.COSE:
            return self.generate_cose_wave(params)
        elif waveform_type == WaveformType.IQ:
            return self.generate_IQ_wave(params)
        elif waveform_type == WaveformType.MULTI_TONE:
            return self.generate_multi_tone(params)
        elif waveform_type == WaveformType.SQUARE:
            return self.generate_square_wave(params)
        elif waveform_type == WaveformType.PULSE:
            return self.generate_pulse_signal(params)
        elif waveform_type == WaveformType.TRIANGLE:
            return self.generate_triangle_wave(params)
        elif waveform_type == WaveformType.CHIRP:
            return self.generate_chirp_signal(params)
        else:
            raise ValueError(f"Unsupported waveform type: {params.waveform_type}")

print("Jupyter signal generator class definition completed!")

Jupyter信号生成器类定义完成！


In [ ]:
# Jupyter optimized visualization and analysis functions
class JupyterSignalAnalyzer:
    """Signal analyzer for Jupyter environment"""
    
    def __init__(self, generator: JupyterSignalGenerator):
        self.generator = generator
        
    def plot_signal_inline(self, t: np.ndarray, signal_data: np.ndarray, 
                          params: SignalParams, show_fft: bool = False):
        """Display signal inline in Jupyter"""
        if show_fft:
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(14, 6))
        
        # Time domain plot / 时域图
        ax1.plot(t, signal_data, 'b-', linewidth=1.5, alpha=0.8)
        ax1.set_title(f'{params.waveform_type.title()} Signal - Time Domain', fontsize=14)
        ax1.set_xlabel('Time (s)', fontsize=12)
        ax1.set_ylabel('Amplitude', fontsize=12)
        ax1.grid(True, alpha=0.3)
        
        # Add statistics / 添加统计信息
        stats_text = (f'Sample Rate: {params.sample_rate} Hz\\n'
                     f'Duration: {params.duration} s\\n'
                     f'Amplitude: {params.amplitude}\\n'
                     f'Max: {np.max(signal_data):.3f}\\n'
                     f'Min: {np.min(signal_data):.3f}\\n'
                     f'Mean: {np.mean(signal_data):.3f}\\n'
                     f'RMS: {np.sqrt(np.mean(signal_data**2)):.3f}')
        
        ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', 
                facecolor='lightblue', alpha=0.8))
        
        if show_fft:
            # Frequency domain plot
            fft_data = np.fft.fft(signal_data)
            freqs = np.fft.fftfreq(len(signal_data), 1/params.sample_rate)
            
            positive_freqs = freqs[:len(freqs)//2]
            magnitude = np.abs(fft_data[:len(fft_data)//2])
            magnitude_db = 20 * np.log10(magnitude + 1e-10)
            
            ax2.plot(positive_freqs, magnitude_db, 'r-', linewidth=1.5, alpha=0.8)
            ax2.set_title('Frequency Domain Analysis (FFT)', fontsize=14)
            ax2.set_xlabel('Frequency (Hz)', fontsize=12)
            ax2.set_ylabel('Magnitude (dB)', fontsize=12)
            ax2.grid(True, alpha=0.3)
            ax2.set_xlim(0, min(5000, params.sample_rate/2))  # Limit display range
        
        plt.tight_layout()
        plt.show()
        
    def compare_waveforms(self, waveform_types: List[str], base_params: SignalParams):
        """Compare different waveform types"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        for i, waveform_type in enumerate(waveform_types[:4]):  # Show max 4 waveforms
            params = SignalParams(
                waveform_type=waveform_type,
                duration=base_params.duration,
                sample_rate=base_params.sample_rate,
                amplitude=base_params.amplitude,
                frequency=base_params.frequency
            )
            
            t, signal_data = self.generator.generate_signal(params)
            
            # Show only first few cycles / 只显示前几个周期
            display_samples = min(int(0.01 * params.sample_rate), len(t))
            t_display = t[:display_samples]
            signal_display = signal_data[:display_samples]
            
            axes[i].plot(t_display * 1000, signal_display, linewidth=2)
            axes[i].set_title(f'{waveform_type.title()} Waveform', fontsize=12)
            axes[i].set_xlabel('Time (ms)', fontsize=10)
            axes[i].set_ylabel('Amplitude', fontsize=10)
            axes[i].grid(True, alpha=0.3)
        
        plt.suptitle('Waveform Types Comparison', fontsize=16)
        plt.tight_layout()
        plt.show()
        
    def plot_frequency_spectrum(self, t: np.ndarray, signal_data: np.ndarray, 
                               params: SignalParams, log_scale: bool = True):
        """Plot frequency spectrum"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Calculate FFT
        fft_data = np.fft.fft(signal_data)
        freqs = np.fft.fftfreq(len(signal_data), 1/params.sample_rate)
        
        # Positive frequency part
        positive_freqs = freqs[:len(freqs)//2]
        magnitude = np.abs(fft_data[:len(fft_data)//2])
        phase = np.angle(fft_data[:len(fft_data)//2])
        
        # Magnitude spectrum
        if log_scale:
            magnitude_db = 20 * np.log10(magnitude + 1e-10)
            ax1.plot(positive_freqs, magnitude_db, 'b-', linewidth=1.5)
            ax1.set_ylabel('Magnitude (dB)', fontsize=12)
        else:
            ax1.plot(positive_freqs, magnitude, 'b-', linewidth=1.5)
            ax1.set_ylabel('Magnitude', fontsize=12)
        
        ax1.set_title('Magnitude Spectrum', fontsize=14)
        ax1.set_xlabel('Frequency (Hz)', fontsize=12)
        ax1.grid(True, alpha=0.3)
        ax1.set_xlim(0, min(5000, params.sample_rate/2))
        
        # Phase spectrum
        ax2.plot(positive_freqs, phase, 'r-', linewidth=1.5)
        ax2.set_title('Phase Spectrum', fontsize=14)
        ax2.set_xlabel('Frequency (Hz)', fontsize=12)
        ax2.set_ylabel('Phase (radians)', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.set_xlim(0, min(5000, params.sample_rate/2))
        
        plt.tight_layout()
        plt.show()
        
    def analyze_signal_statistics(self, signal_data: np.ndarray, params: SignalParams):
        """Analyze signal statistical characteristics"""
        stats = {
            'Signal Length': len(signal_data),
            'Duration': f"{params.duration:.3f} s",
            'Sample Rate': f"{params.sample_rate} Hz",
            'Maximum': f"{np.max(signal_data):.6f}",
            'Minimum': f"{np.min(signal_data):.6f}",
            'Mean': f"{np.mean(signal_data):.6f}",
            'RMS': f"{np.sqrt(np.mean(signal_data**2)):.6f}",
            'Std Dev': f"{np.std(signal_data):.6f}",
            'Peak-to-Peak': f"{np.max(signal_data) - np.min(signal_data):.6f}",
            'Crest Factor': f"{np.max(np.abs(signal_data)) / np.sqrt(np.mean(signal_data**2)):.3f}",
            'Form Factor': f"{np.sqrt(np.mean(signal_data**2)) / np.mean(np.abs(signal_data)):.3f}"
        }
        
        print("=== Signal Statistical Analysis ===")
        for key, value in stats.items():
            print(f"{key:15s}: {value}")
        
        return stats

print("Signal analyzer created successfully!")

信号分析器创建完成！
